In [ ]:
print("Welcome to Chroma DB session")

## Import important Library and some docoment for practice

In [ ]:
!pip -q install chromadb openai langchain tiktoken

In [ ]:
!pip show chromadb

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [ ]:
!unzip -q new_articles.zip -d new_articles

## Setting up Environment

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "sushil-xxxxxxxxxxxxxxxxxxxxxxxx"

## Import some libraries

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

## Load data

In [ ]:
loader = DirectoryLoader("/chroma_content/news_articles/", glob = "./*.txt", loader_cls= TextLoader)

In [ ]:
document = loader.load()

In [ ]:
document

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

In [ ]:
text

In [ ]:
len(text)

In [ ]:
text[1]

In [ ]:
text[2]

## Creating DB

In [ ]:
from langchain import embeddings
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=text,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("How  big investment microsoft raise?")

In [ ]:
len(docs)

In [ ]:
docs

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.search_type

In [ ]:
retriever.search_kwargs

## Make a chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm=OpenAI()

In [ ]:
llm

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## proper document retevial function
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "money raised by microsoft?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

## Deleteing the DB


In [ ]:
!zip -r db.zip ./db

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory of db
!rm -rf db/